[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab1/p1.ipynb)

# Práctica 1: Redes neuronales desde cero con Numpy

En esta práctica vamos a familiarizarnos con el algoritmo de descenso de gradiente estocástico con backpropagation. Para ello, vamos a desarrollar un clasificador que utilice una red neuronal para modelar los datos y hacer predicciones. Esto requiere que diseñemos la red neuronal y que hagamos el entrenamiento para que las predicciones sean correctas para el conjunto de datos dado. Implementaremos todo usando `numpy`.


# Pre-requisitos

## Instalar paquetes

Para esta práctica solo necesitaremos `numpy` (y `pandas`, `sklearn` y `seaborn` para cargar el conjunto de datos)

In [ ]:
#!pip install numpy seaborn pandas
import numpy as np
import seaborn as sns
import pandas as pd

## Creación de una neurona artificial con Numpy

Una neurona artificial consta de dos partes diferenciadas. En primer lugar, la unidad realiza una suma de todas sus entradas (más una componente de *bias*), cada una de ellas ponderada por un peso. Estos pesos (y el *bias*) serán los que se modifiquen para conseguir que la neurona dé la salida adecuada para nuestro problema para cada combinación de entradas del conjunto de datos de entrada.

Dado un vector de entrada $\mathbf{x}$ con $d$ componentes y un vector de pesos $\mathbf{w}$, esta primera parte de la neurona calculará un único valor escalar de salida que llamaremos $z$ de la siguiente forma:

$$
z = \sum \limits_{i=0}^{d} \mathbf{x}_i\mathbf{w}_i + bias
$$

### Notación vectorial

Para simplificar la notación, podemos representar todos los pesos de una neurona como un vector. Al hacerlo así, la suma ponderada de las entradas será el producto escalar del vector $\mathbf{x}$ de entrada y el vector $\mathbf{w}$ de pesos. Teniendo en cuenta que asumiremos que el vector de entrada $\mathbf{x}$ es un vector fila de dimensiones 1 x $d$, declararemos el vector de pesos $\mathbf{w}$ con las mismas dimensiones y podremos representar el producto escalar $\mathbf{x} · \mathbf{w}$ como el producto matricial $\mathbf{x}\mathbf{w}^T$. Esto es ventajoso para poder procesar varios vectores de entrada utilizando la misma operación.

$$
z = \sum \limits_{i=0}^{d} \mathbf{x}_i\mathbf{w}_i + bias = \mathbf{x} · \mathbf{w} + bias = \mathbf{x} \mathbf{w}^T + bias
$$

Completa la siguiente celda para calcular $z$.

In [ ]:
x = np.array([1, 2, 8, -4]).reshape((1,4)) # vector de entrada
w = np.array([0.1, -0.8, 0.3, 0.2]).reshape((1,4)) # vector de pesos
bias = 0.1

# TODO - completa esta línea usando np.matmul
z = 

# verificación del resultado
np.testing.assert_almost_equal(0.2, z, err_msg='Revisa tu implementación')

### Función de activación

Tras este primer paso, la salida $z$ será una combinación lineal de las entradas. Si concatenamos varias neuronas así definidas, el resultado seguirá siendo una combinación lineal de las entradas, lo cual no es muy útil dado que se podría obtener el mismo resultado con una sola neurona. Es por ello que necesitamos que cada neurona tenga una segunda parte que introduzca una no-linealidad. Es lo que denominamos *función de activación*. En este ejemplo tomaremos como función de activación la función sigmoide, definida como:

$$sigmoide(x) = \dfrac{1}{1+e^{-x}}$$

Completa a continuación el código para calcular la sigmoide de un escalar $x$:

In [ ]:
def sigmoide(x):
    #TODO - Completa la siguiente línea
    return 

# verificación del resultado
np.testing.assert_almost_equal(0.54983399, sigmoide(z))

En este caso, tomaremos como entrada de la función sigmoide la salida del paso anterior, $z$. Por tanto, la salida $y$ de la neurona artificial será un escalar de la siguiente forma:

$$y = sigmoide(z) = \dfrac{1}{1+e^{-(\mathbf{x}\mathbf{w}^T + bias)}}$$

Puedes ver el esquema general de una neurona artificial en este diagrama:

<img src="./img/neural-model.png" alt="Diagrama de una neurona artificial" width="700"/>

Completa el código de esta función realizar el *forward pass* de una neurona artificial con función de activación sigmoide, es decir, para calcular la salida a partir del vector de entrada, el vector de pesos y el valor de bias:

In [ ]:
def neurona_forward(x, w, bias):
    #TODO - Completa la siguiente línea
    return 

# verificación
np.testing.assert_almost_equal(0.54983399, neurona_forward(x, w, bias))

## Red feed-forward

Con este montaje, si utilizásemos descenso de gradiente para aprender el vector $\mathbf{w}$ que haga las predicciones correctas para un conjunto de datos dado, estaríamos entrenando un modelo de regresión logística. Sin embargo, la potencia de las redes neuronales reside en la posibilidad de combinar muchas de estas unidades para poder modelar funciones mucho más complejas, por lo que vamos a construir una red que utilice varias unidades.

El modo más sencillo de organizar varias neuronas es formando una red *feed-forward*. Para ello, primero agruparemos varias neuronas formando una *capa*. Si la salida de una neurona era un escalar $y$, la salida de una capa será un vector $\mathbf{y}$ con tantas componentes como unidades tenga la capa. Igualmente, si en una neurona teníamos un vector $\mathbf{w}$ de pesos, en una capa tendremos una matriz $\mathbf{W}$ de pesos, en la que cada fila corresponderá con el vector de pesos de una neurona de la capa. También tendremos un vector $\mathbf{b}$, donde cada componente será el *bias* de una neurona de la capa.

Podemos aprovechar las operaciones de matrices y vectores de NumPy para hacer el *forward pass* de toda una capa de una sola vez. Completa el siguiente código para obtener una función que realice el *forward pass* de una capa de neuronas artificiales con activación sigmoide tomando como entradas el vector de entrada $\mathbf{x}$, una matriz de pesos $\mathbf{W}$ y un vector $\mathbf{b}$.

In [ ]:
# Asegúrate de que tu implementación de sigmoide puede tomar como entrada un vector. (La función np.exp de NumPy te puede resultar útil, porque puede tomar como entrada un escalar y devolver un escalar, pero también puede tomar un vector y devolver otro vector).
np.testing.assert_almost_equal([0.549834, 0.98201379], sigmoide(np.array([z[0][0],4])))

def capa_forward(x, W, b):
    # TODO - Completa el código. La versatilidad de NumPy hace que esto sea muy sencillo
    return 

# verificación
np.testing.assert_almost_equal(np.array([[0.549834], [0.05732418]]), capa_forward(np.vstack((x, np.array([-1, 3, -2, 2]))), w, np.array([bias, -0.1]).reshape((2,1))))

Ahora podemos concatenar capas de neuronas con facilidad. Vamos a crear una red de tres capas:
 1. La capa $C_0$ consta de 5 unidades. Recibe como entrada el vector $\mathbf{x}$ y produce como salida el vector $\mathbf{h_0}$. Tiene una matriz de pesos $\mathbf{W_0}$ y un vector de bias $\mathbf{b_0}$.
 1. La capa $C_1$ consta de 3 unidades. Recibe como entrada el vector $\mathbf{h_0}$ y produce como salida el vector $\mathbf{h_1}$. Tiene una matriz de pesos $\mathbf{W_1}$ y un vector de bias $\mathbf{b_1}$.
 1. La capa $C_2$ consta de 1 unidad. Recibe como entrada el vector $\mathbf{h_1}$ y produce como salida el vector $\mathbf{y}$. Tiene una matriz de pesos $\mathbf{W_2}$ y un vector de bias $\mathbf{b_2}$.
 
<img src="./img/lab1-red.png" alt="Diagrama de una neurona artificial" width="700"/>
 
Completa la siguiente celda para calcular la salida de la red.

In [ ]:
# Inicialización de pesos
np.random.seed(1234567) # Fijamos la semilla para que los números aleatorios salgan igual en cada ejecución para poder verificar los resultados
# TODO - Completa las dimensiones de las matrices
W0 = np.random.rand(5, x.shape[1]) - 0.5
b0 = np.random.rand(1, 5) - 0.5
W1 = np.random.rand(, ) - 0.5
b1 = np.random.rand(, ) - 0.5
W2 = np.random.rand(, ) - 0.5
b2 = np.random.rand(, ) - 0.5


# Función auxiliar que utiliza la red neuronal para hacer una predicción
def calcula_prediccion(x):
    # TODO - Calcula las salidas de cada capa
    h0 = 
    h1 = 
    y = 
    return y

# verificación
np.testing.assert_almost_equal(0.34535528, calcula_prediccion(x))

# Hacer predicciones con la red sobre un conjunto de datos
Ahora que sabemos definir una red, probemos qué predicciones hace sobre un conjunto de datos. Cargaremos el conjunto `titanic` (que busca predecir la supervivencia de pasajeros del Titanic a partir de sus características) usando la librería `seaborn`. Con esos datos, calcularemos la salida predicha para cada entrada y la compararemos con la etiqueta que se espera.

In [ ]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler

def load_titanic():
    # Cargamos el dataset Titanic desde seaborn
    df = sns.load_dataset('titanic')

    # 1️⃣ Selección de variables relevantes y limpieza
    # Columnas que vamos a usar
    cols = ['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'alone']
    df = df[cols].copy()

    # Eliminamos filas con valores faltantes
    df = df.dropna(subset=['age', 'embarked', 'fare'])

    # 2️⃣ Separar etiquetas y características
    y = df['survived'].to_numpy().astype(np.float32)        # etiquetas como float
    X = df.drop(columns=['survived'])

    # 3️⃣ One-hot encoding para todas las variables categóricas
    categorical_cols = ['pclass', 'sex', 'embarked', 'alone']
    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)  # drop_first=True evita multicolinealidad

    # 4️⃣ Variables numéricas
    numeric_cols = ['age', 'sibsp', 'parch', 'fare']
    X_numeric = X_encoded[numeric_cols + [c for c in X_encoded.columns if c not in numeric_cols]]
    scaler = StandardScaler()
    X_numeric[numeric_cols] = scaler.fit_transform(X_numeric[numeric_cols])

    # 5️⃣ Convertir a numpy arrays
    X_np = X_numeric.to_numpy().astype(np.float32)
    y_np = y.reshape(-1, 1).astype(np.float32)  # reshape para que sea (n_samples,1)

    return X_np, y_np

In [ ]:
X, y = load_titanic()

# TODO - Adapta las dimensiones de los parámetros de la capa de entrada (C0) para que coincidan con el tamaño de los vectores del conjunto de datos, que tienen 24 componentes
W0 = np.random.rand(, ) - 0.5
b0 = np.random.rand(, )


# Función auxiliar que, dada una lista de vectores de entrada, calcula la tasa de aciertos de la red
def calcula_tasa_aciertos(ejemplos_x, ejemplos_y):
    num_aciertos = 0
    num_elems = 0
    for x, label in zip(ejemplos_x, ejemplos_y):
        y_pred = calcula_prediccion(x)

        # Actualizamos los contadores de elementos y de aciertos
        num_elems += 1
        if ((y_pred > 0.5) and (label==1)) or ((y_pred <= 0.5) and (label==0)):
            num_aciertos += 1

    return num_aciertos / float(num_elems)
        
print('La tasa de acierto es del', calcula_tasa_aciertos(X[:100], y[:100]))

Previsiblemente, la red no funciona bien dado que las matrices de pesos contienen vectores aleatorios. Para que las predicciones mejoren, tenemos que ajustar nuestro modelo al conjunto de datos, es decir, encontrar valores para los parámetros ($\mathbf{W_0}$, $\mathbf{b_0}$, $\mathbf{W_1}$, $\mathbf{b_1}$, $\mathbf{W_2}$, $\mathbf{b_2}$) que arrojen buenas predicciones.

# Entrenamiento de la red

## La función de coste
Para entrenar la red utilizaremos un proceso de optimización. En primer lugar, debemos definir qué función queremos optimizar. Necesitamos una función que, dada una combinación de parámetros de la red, devuelva un valor alto cuando las predicciones con esos parámetros sean malas y un valor bajo cuando estas sean buenas. Esto es lo que denominamos la **función de coste** ($J$). Definiremos una **función de pérdida** ($\mathcal{L}$) que reciba como entrada una predicción y una etiqueta real y nos indique cómo de desacertada es la predicción. En este caso utilizaremos la entropía cruzada binaria, descrita como:

$$ \mathcal{L}(y_{pred},y_{etiqueta}) = - y_{etiqueta} \log(y_{pred}) - (1-y_{etiqueta})  \log(1-y_{pred}) $$

La función de coste ($J$) será la media de la función de pérdida en los $m$ ejemplos del conjunto de entrenamiento:
$$ J(\mathbf{W},\mathbf{b}) = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(y_{pred}^{(i)}, y_{etiqueta}^{(i)})$$

Si minimizamos $J$, nuestras predicciones serán mejores. Para minimizar $J$ utilizaremos **descenso de gradiente**: haremos sucesivos pasos en los que calcularemos el gradiente de $J$ respecto a los distintos parámetros ($\mathbf{W},\mathbf{b}$) y actualizaremos los parámetros en la dirección del gradiente, con la esperanza de que el siguiente paso obtenga un valor de $J$ menor. Repetiremos este proceso durante un número fijo de pasos.

Por tanto, el algoritmo que debemos aplicar es el siguiente:
 1. Calcular la pérdida de las predicciones con los valores actuales de $\mathbf{W}$ y $\mathbf{b}$
 1. Calcular el gradiente respecto $\mathbf{W}$ y $\mathbf{b}$.
 1. Actualizar $\mathbf{W}$ y $\mathbf{b}$ en la dirección de sus gradientes respectivos.

## Gradientes
El segundo paso nos obliga a ser capaces de calcular el gradiente de $J(\mathbf{W},\mathbf{b})$ respecto a cada uno de los parámetros, es decir, la derivada parcial de $J(\mathbf{W},\mathbf{b})$ respecto a cada parámetro. Para ello iremos propagando el gradiente hacia atrás, calculando a cada paso el gradiente en el nodo anterior a partir de los posteriores. Calculemos, por ejemplo, el gradiente de $J(\mathbf{W},\mathbf{b})$ respecto a $z_2$:

$$ \frac{\partial J(\mathbf{W},\mathbf{b})}{\partial z_2} = \frac{\partial (\frac{1}{m} \sum_{i=1}^m \mathcal{L}(y_{pred}^{(i)}, y_{etiqueta}^{(i)}))}{\partial z_2} = \frac{1}{m} \sum_{i=1}^m \frac{\partial \mathcal{L}(y_{pred}^{(i)}, y_{etiqueta}^{(i)})}{\partial z_2} $$

Aplicando la regla de la cadena ([ver el desarrollo completo](./lab2-gradientes.pdf)) obtenemos las siguientes fórmulas para los gradientes:

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial b_2} = \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial z_2}$$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial W_2} = \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial z_2} h_1^T $$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial h_1} = \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial z_2} W_2^T $$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial z_1} = \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial h_1} h_1 (1 - h_1) = dLdz1$$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial b_1} = \sum_{i=1}^{5} dJdz1_i $$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial W_1} = diag(dLdz1) h_0^T $$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial h_0} = diag(dLdz1) W_1 $$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial z0} = \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial h_0} h_0 (1 - h_0) = dLdz0$$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial b_0} = \sum_{i=1}^{5} dLdz0_i $$

$$ \frac{\partial \mathcal{L}(y_{pred}, y_{etiqueta})}{\partial W_0} = diag(dLdz0) x^T $$

Conociendo estas fórmulas podemos adaptar nuestra función para que, además de calcular la predicción, devuelva los gradientes de la función de pérdida respecto a las variables involucradas.

In [ ]:
def calcula_y_propaga(x, y_etiqueta):
    ''' Devuelve la salida predicha y, el valor de la función de pérdida y un diccionario con los gradientes de las variables
    '''
    x = np.expand_dims(x, axis=0)
    
    # Cálculo de la salida
    # TODO - Rellena las siguientes líneas
    z0 = 
    h0 = 
    z1 = 
    h1 = 
    z2 = 
    y = 
    
    #Backpropagation
    dLdz2 = y - y_etiqueta
    dLdb2 = dLdz2
    dLdW2 = dLdz2 * h1
    dLdh1 = np.matmul(dLdz2.T, W2)
    dLdz1 = dLdh1 * h1 * (1 - h1)
    dLdb1 = dLdz1
    dLdW1 = np.matmul(dLdz1.T, h0)
    dLdh0 = np.matmul(dLdz1, W1)
    dLdz0 = dLdh0 * h0 * (1 - h0)
    dLdb0 = dLdz0
    dLdW0 = np.matmul(dLdz0.T, x)
    
    # Los gradientes tienen que tener la misma forma que las variables respecto a las que se toman
    assert(dLdz2.shape==z2.shape)
    assert(dLdb2.shape==b2.shape)
    assert(dLdW2.shape==W2.shape)
    assert(dLdh1.shape==h1.shape)
    assert(dLdz1.shape==z1.shape)
    assert(dLdb1.shape==b1.shape)
    assert(dLdW1.shape==W1.shape)
    assert(dLdh0.shape==h0.shape)
    assert(dLdz0.shape==z0.shape)
    assert(dLdb0.shape==b0.shape)
    assert(dLdW0.shape==W0.shape)
    
    # Preparación del diccionario de gradientes
    gradientes = {}
    gradientes["b2"] = dLdb2
    gradientes["b1"] = dLdb1
    gradientes["b0"] = dLdb0
    gradientes["W2"] = dLdW2
    gradientes["W1"] = dLdW1
    gradientes["W0"] = dLdW0
    
    # Pérdida
    # TODO - Rellena la siguiente línea
    perdida = 
    return y, perdida.flatten(), gradientes

# verificación
prediccion, perdida, gradientes = calcula_y_propaga(X[0], y[0])
np.testing.assert_almost_equal(0.3463054, prediccion[0])
np.testing.assert_almost_equal(0.4251149366689026, perdida[0])

## Bucle de aprendizaje
Una vez podemos calcular los gradientes respecto a las variables para cada ejemplo del conjunto de entrenamiento ya podemos implementar el bucle de entrenamiento.

Completa esta función para que realice `num_pasos` de entrenamiento en los cuales se haga lo siguiente:
 1. Calcular las predicciones, pérdidas y gradientes para cada elemento del conjunto de entrenamiento
 1. Calcular el valor promedio de cada gradiente
 1. Utilizar dichos promedios para actualizar las variables
Llevaremos también cuenta de la tasa de aciertos y del valor de la función de coste a cada paso.

In [ ]:
def entrena(ejemplos, etiquetas, num_pasos, learning_rate = 0.01):
    paso = 0
    while paso < num_pasos:
        num_aciertos = 0
        num_elems = 0
        perdida_total = 0
        for x, label in zip(ejemplos, etiquetas):
            # TODO - Completa la siguiente línea
            y_pred, perdida, gradientes = 

            # Actualizamos los contadores de elementos y de aciertos
            num_elems += 1
            if ((y_pred > 0.5) and (label==1)) or ((y_pred <= 0.5) and (label==0)):
                num_aciertos += 1
                
            # Actualizamos el acumulador de la pérdida_total
            perdida_total += perdida
            
            # Actualizamos las variables en la dirección de su gradiente
            global W2, W1, W0, b2, b1, b0
            # TODO - Completa las siguientes líneas
            W2 = W2 - learning_rate * gradientes['W2']
            W1 = 
            W0 = 
            b2 = 
            b1 = 
            b0 = 
            
        tasa_aciertos = num_aciertos / float(num_elems)
        perdida = perdida_total / float(num_elems)
        print(f'Epoch: {paso}/{num_pasos}: Pérdida: {perdida} Aciertos: {tasa_aciertos}')
        paso += 1

Para probar nuestro algoritmo de entrenamiento, tomemos solo los 100 primeros elementos del conjunto de datos y entrenemos el modelo para que se ajuste a dichos datos.

In [ ]:
tamano_lote = 100
entrena(X[:tamano_lote], y[:tamano_lote], 5000)

Implementar el bucle de entrenamiento es una tarea complicada porque cada iteración requiere hacer el *forward propagation* y el *backpropagation* lo que, a su vez, implica llevar cuenta de resultados parciales para calcular cada uno de los gradientes con los que actualizar los parámetros.

Además de la complejidad de organizar el código que lleve a cabo dicho algoritmo, se deben tener en cuenta los siguientes factores:
 - El código debería permitir redes con cualquier número de capas las cuales, a su vez, deberán contener un número arbitrario de unidades. Incluso sería ideal que permitiese arquitecturas distinta de la red *feed forward*.
 - El uso de memoria debe ser eficiente, no almacenando más resultados de los necesarios para cada cómputo.
 - Del mismo modo, debería evitarse realizar operaciones innecesarias y paralelizar las que sea posible.
 - El código debería ser extensible para permitir otras funciones de coste.
 
Por todo ello es por lo que habitualmente se utilizan frameworks especializados como TensorFlow para trabajar con redes neuronales. Este tipo de software nos permite definir modelos y entrenarlos de manera eficiente sin que debamos preocuparnos de calcular derivadas ni de los detalles de implementación.